In [1]:
!git clone 'https://github.com/aakashvardhan/s7-in-depth-practice.git'

Cloning into 's7-in-depth-practice'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 68 (delta 26), reused 60 (delta 18), pack-reused 0
Receiving objects: 100% (68/68), 170.53 KiB | 674.00 KiB/s, done.
Resolving deltas: 100% (26/26), done.


In [2]:
import sys
sys.path.append('/content/s7-in-depth-practice')
sys.path.append('/content/s7-in-depth-practice/models')

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model_utils import model_summary, test_model_sanity, sgd_optimizer
from models.model import model2

from utils import train, test, get_device, plt_fig

In [4]:
random_rotations = 7
img_size = (28,28)

# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomResizedCrop(img_size, (0.80, 1.00)),
                                       transforms.RandomRotation(random_rotations),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [5]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 342060234.85it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 102224214.20it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 178038136.09it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20980758.56it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
import os
os.cpu_count()

2

In [7]:
# CUDA?
cuda = get_device()

# For reproducibility
torch.manual_seed(1)

if torch.cuda.is_available():
    torch.cuda.manual_seed(1)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
model2 = model2().to(cuda)

In [9]:
model_summary(model2)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
         ConvBlock-5            [-1, 8, 26, 26]               0
            Conv2d-6           [-1, 16, 24, 24]           1,152
              ReLU-7           [-1, 16, 24, 24]               0
       BatchNorm2d-8           [-1, 16, 24, 24]              32
           Dropout-9           [-1, 16, 24, 24]               0
        ConvBlock-10           [-1, 16, 24, 24]               0
           Conv2d-11           [-1, 16, 22, 22]           2,304
             ReLU-12           [-1, 16, 22, 22]               0
      BatchNorm2d-13           [-1, 16, 22, 22]              32
          Dropout-14           [-1, 16,

In [10]:
optimizer = sgd_optimizer(model2, 0.01)
EPOCHS = 15
for epoch in range(1,EPOCHS+1):
    print("EPOCH:", epoch)
    train(model2, cuda, train_loader, optimizer, epoch)
    test(model2, cuda, test_loader)

EPOCH: 1


Loss=0.20069919526576996 Batch_id=468 Accuracy=83.73: 100%|██████████| 469/469 [00:34<00:00, 13.41it/s]



Test set: Average loss: 0.1097, Accuracy: 9689/10000 (96.89%)

EPOCH: 2


Loss=0.2222498059272766 Batch_id=468 Accuracy=96.51: 100%|██████████| 469/469 [00:31<00:00, 14.72it/s]



Test set: Average loss: 0.0541, Accuracy: 9859/10000 (98.59%)

EPOCH: 3


...


Test set: Average loss: 0.0213, Accuracy: 9935/10000 (99.35%)

EPOCH: 14


Loss=0.04589877650141716 Batch_id=468 Accuracy=98.50: 100%|██████████| 469/469 [00:32<00:00, 14.63it/s]



Test set: Average loss: 0.0243, Accuracy: 9920/10000 (99.20%)

EPOCH: 15


Loss=0.01317374873906374 Batch_id=468 Accuracy=98.54: 100%|██████████| 469/469 [00:31<00:00, 14.81it/s]



Test set: Average loss: 0.0224, Accuracy: 9927/10000 (99.27%)

